In [6]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3

# ---------- BASE DE DATOS ----------
def conectar_db():
    conn = sqlite3.connect("estudiantes.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS estudiantes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            cedula TEXT,
            nombre TEXT NOT NULL,
            edad INTEGER,
            correo TEXT
        )
    """)
    # Intentar agregar columna cedula si no existía
    try:
        cursor.execute("ALTER TABLE estudiantes ADD COLUMN cedula TEXT")
    except sqlite3.OperationalError:
        pass  # Ya existe
    conn.commit()
    conn.close()

# ---------- FUNCIONES PRINCIPALES ----------
def agregar_estudiante():
    try:
        cedula = entry_cedula.get().strip()
        nombre = entry_nombre.get().strip()
        edad = entry_edad.get().strip()
        correo = entry_correo.get().strip()

        if not cedula or not nombre or not edad or not correo:
            messagebox.showwarning("Advertencia", "Todos los campos son obligatorios.")
            return

        # Validar edad numérica
        try:
            edad = int(edad)
        except ValueError:
            messagebox.showerror("Error", "La edad debe ser un número.")
            return

        conn = sqlite3.connect("estudiantes.db")
        cursor = conn.cursor()

        # Verificar que la cédula no esté repetida
        cursor.execute("SELECT * FROM estudiantes WHERE cedula=?", (cedula,))
        if cursor.fetchone():
            messagebox.showwarning("Duplicado", "La cédula ya está registrada.")
            conn.close()
            return

        cursor.execute("INSERT INTO estudiantes (cedula, nombre, edad, correo) VALUES (?, ?, ?, ?)",
                       (cedula, nombre, edad, correo))
        conn.commit()
        conn.close()

        limpiar_campos()
        mostrar_estudiantes()
        messagebox.showinfo("Éxito", "Estudiante agregado correctamente.")

    except Exception as e:
        messagebox.showerror("Error", f"Ocurrió un problema: {e}")

def mostrar_estudiantes():
    for row in tree.get_children():
        tree.delete(row)
    conn = sqlite3.connect("estudiantes.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM estudiantes")
    for fila in cursor.fetchall():
        tree.insert("", tk.END, values=fila)
    conn.close()

def eliminar_estudiante():
    seleccionado = tree.selection()
    if not seleccionado:
        messagebox.showwarning("Advertencia", "Seleccione un estudiante para eliminar.")
        return
    id_estudiante = tree.item(seleccionado)["values"][0]

    conn = sqlite3.connect("estudiantes.db")
    cursor = conn.cursor()
    cursor.execute("DELETE FROM estudiantes WHERE id=?", (id_estudiante,))
    conn.commit()
    conn.close()

    mostrar_estudiantes()
    messagebox.showinfo("Eliminado", "Estudiante eliminado correctamente.")

def limpiar_campos():
    entry_cedula.delete(0, tk.END)
    entry_nombre.delete(0, tk.END)
    entry_edad.delete(0, tk.END)
    entry_correo.delete(0, tk.END)

# ---------- VALIDACIÓN DE ENTRADA ----------
def solo_numeros(caracter):
    """Permite solo números en el campo de edad y cédula"""
    return caracter.isdigit() or caracter == ""

# ---------- INTERFAZ GRÁFICA ----------
ventana = tk.Tk()
ventana.title("📘 Registro de Estudiantes")
ventana.geometry("700x480")
ventana.configure(bg="#E8F0F2")

# ---------- ESTILOS ----------
style = ttk.Style()
style.theme_use("clam")

style.configure("Treeview",
                background="#FAFAFA",
                foreground="#000000",
                rowheight=25,
                fieldbackground="#FAFAFA")
style.map("Treeview", background=[("selected", "#A7C7E7")])
style.configure("Treeview.Heading",
                background="#2F4F4F",
                foreground="white",
                font=("Arial", 10, "bold"))
style.configure("TLabel", background="#E8F0F2", font=("Segoe UI", 10))
style.configure("TLabelframe", background="#E8F0F2", font=("Segoe UI", 11, "bold"))
style.configure("TLabelframe.Label", foreground="#2F4F4F", font=("Segoe UI", 11, "bold"))

def crear_boton(texto, comando, color):
    return tk.Button(frame_botones,
                     text=texto,
                     command=comando,
                     bg=color,
                     fg="white",
                     activebackground="#34495E",
                     activeforeground="white",
                     relief="flat",
                     font=("Segoe UI", 10, "bold"),
                     padx=10,
                     pady=5,
                     cursor="hand2")

# ---------- SECCIONES ----------
frame_titulo = tk.Frame(ventana, bg="#2F4F4F")
frame_titulo.pack(fill="x")

titulo = tk.Label(frame_titulo,
                  text="📋 REGISTRO DE ESTUDIANTES",
                  bg="#2F4F4F",
                  fg="white",
                  font=("Segoe UI", 14, "bold"),
                  pady=10)
titulo.pack()

frame_form = ttk.LabelFrame(ventana, text="Datos del Estudiante")
frame_form.pack(fill="x", padx=15, pady=10)

validacion_num = ventana.register(solo_numeros)

# Campos
ttk.Label(frame_form, text="Cédula:").grid(row=0, column=0, padx=5, pady=5, sticky="e")
entry_cedula = ttk.Entry(frame_form, width=30, validate="key", validatecommand=(validacion_num, "%P"))
entry_cedula.grid(row=0, column=1, padx=5, pady=5)

ttk.Label(frame_form, text="Nombre:").grid(row=1, column=0, padx=5, pady=5, sticky="e")
entry_nombre = ttk.Entry(frame_form, width=30)
entry_nombre.grid(row=1, column=1, padx=5, pady=5)

ttk.Label(frame_form, text="Edad:").grid(row=2, column=0, padx=5, pady=5, sticky="e")
entry_edad = ttk.Entry(frame_form, width=30, validate="key", validatecommand=(validacion_num, "%P"))
entry_edad.grid(row=2, column=1, padx=5, pady=5)

ttk.Label(frame_form, text="Correo:").grid(row=3, column=0, padx=5, pady=5, sticky="e")
entry_correo = ttk.Entry(frame_form, width=30)
entry_correo.grid(row=3, column=1, padx=5, pady=5)

# ---------- BOTONES ----------
frame_botones = tk.Frame(ventana, bg="#E8F0F2")
frame_botones.pack(fill="x", padx=15, pady=10)

crear_boton("➕ Agregar", agregar_estudiante, "#27AE60").pack(side="left", padx=5)
crear_boton("🗑️ Eliminar", eliminar_estudiante, "#E74C3C").pack(side="left", padx=5)
crear_boton("🔄 Actualizar", mostrar_estudiantes, "#3498DB").pack(side="left", padx=5)
crear_boton("🧹 Limpiar", limpiar_campos, "#9B59B6").pack(side="left", padx=5)

# ---------- TABLA ----------
frame_tabla = ttk.Frame(ventana)
frame_tabla.pack(fill="both", expand=True, padx=15, pady=10)

tree = ttk.Treeview(frame_tabla, columns=("ID", "Cédula", "Nombre", "Edad", "Correo"), show="headings")
tree.heading("ID", text="ID")
tree.heading("Cédula", text="Cédula")
tree.heading("Nombre", text="Nombre")
tree.heading("Edad", text="Edad")
tree.heading("Correo", text="Correo")
tree.pack(fill="both", expand=True)

# ---------- INICIO ----------
conectar_db()
mostrar_estudiantes()
ventana.mainloop()